In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 62.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 67.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv")

In [5]:
df.head()

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.


In [6]:
print("findings = ",df["findings"][0])
print("\n")
print("impression = ",df["impression"][0])

findings =  The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no XXXX of a pleural effusion. There is no evidence of pneumothorax.


impression =  Normal chest x-XXXX.


In [7]:
import pandas as pd

# Assume df is your DataFrame containing 'findings' and 'impression' columns
df['findings'] = df['findings'].str.replace(r'\bXXXX\b', '', regex=True).str.strip()
df['impression'] = df['impression'].str.replace(r'\bXXXX\b', '', regex=True).str.strip()


In [8]:
df["findings"] = df["findings"].fillna("")
df["impression"] = df["impression"].fillna("")

In [9]:
df["combined_text"] = df["findings"] + " " + df["impression"]
df["combined_text"] = df["combined_text"].astype(str)

In [10]:
# Filter out rows where 'combined_text' is empty or contains only whitespace
df = df[df['combined_text'].str.strip() != '']


In [11]:
print("findings = ",df["findings"][0])
print("\n")
print("impression = ",df["impression"][0])
print("\n")
print("combined_text = ", df["combined_text"][0])


findings =  The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no  of a pleural effusion. There is no evidence of pneumothorax.


impression =  Normal chest x-.


combined_text =  The cardiac silhouette and mediastinum size are within normal limits. There is no pulmonary edema. There is no focal consolidation. There are no  of a pleural effusion. There is no evidence of pneumothorax. Normal chest x-.


In [12]:
from sentence_transformers import SentenceTransformer

2025-05-19 19:36:27.526073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747683387.900612      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747683387.985232      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
embeddings = model.encode(df['combined_text'].tolist())

Batches:   0%|          | 0/120 [00:00<?, ?it/s]

In [15]:
import chromadb
from chromadb.config import Settings

In [18]:
# Initialize the persistent client with the desired directory
client = chromadb.PersistentClient(path="/kaggle/working/chroma_db")


In [20]:
collection = client.get_or_create_collection(name = "iu_xray_report_beta1")

In [22]:
collection.add(
    documents = df["combined_text"].tolist(),
    embeddings = embeddings,
    metadatas = [{"index":idx} for idx in df.index],
    ids = [f"doc_{idx}" for idx in df.index]
)


In [23]:
query_text = "Atelectasis, Pleural Effusion"
results = collection.query(
    query_texts = [query_text],
    n_results = 5
)

for doc in results['documents'][0]:
    print(doc)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 24.6MiB/s]


 Streaky opacities in the right lower lobe most suggestive of atelectasis, differential diagnosis includes atypical infection. No definite pleural effusion seen. Heart size within normal limits, no typical findings of pulmonary edema.
Minimal subsegmental atelectasis posteriorly. No pleural effusions or pneumothoraces. Heart and mediastinum of normal size and contour. No focal lung opacity, pleural effusion of pneumothorax.
 Streaky bibasilar opacities most suggestive of atelectasis with hypoventilation. Heart size mildly enlarged for technique, stable mediastinal contours. No definite pleural effusion seen, no typical findings of pulmonary edema.
Moderate sized right loculated pleural effusion with right lower lobe atelectasis. Normal cardiac contour with atherosclerotic changes throughout the aorta. Clear left lung . 1. Moderate right pleural effusion.
Patchy subsegmental atelectasis is seen bibasilar region, no evidence of pneumothorax or pleural effusion is present. The cardiomedia

RE# STORING data in new session****

In [ ]:
# import chromadb

# # Re-initialize the persistent client
# client = chromadb.PersistentClient(path="/kaggle/working/chroma_db")

# # Access your existing collection
# collection = client.get_or_create_collection(name="iu_xray_report_beta1")
